In [ ]:
nm |>
    st_md_to_sf() |>
    st_write("kml_metadata/Friuli-Venezia Giulia/original/osmer.kml")


Writing layer `osmer' to data source 
  `kml_metadata/Friuli-Venezia Giulia/original/osmer.kml' using driver `KML'
Writing 237 features with 8 fields and geometry type Point.


In [ ]:
source("notebooks/ds_regionali/FVG/procedures/load_daily.R")
om <- query_checkpoint_meta("FVG", "merged", conns$data) |> collect()
nm <- load_osmer_meta()


In [ ]:
by_code <- om |>
    select(user_code, sensor_key) |>
    inner_join(nm, by = c("user_code"), relationship = "one-to-one")


In [ ]:
st_join(
    om |> anti_join(by_code, by = "sensor_key") |> st_md_to_sf(),
    nm |> st_md_to_sf(),
    st_is_within_distance,
    dist = 5000,
    left = FALSE
) |>
    st_drop_geometry() |>
    mutate(strSym = stringdist::stringsim(str_to_lower(name.x), str_to_lower(name.y), "jw")) |>
    select(name.x, name.y, strSym, user_code.x, user_code.y) |>
    arrange(name.x, desc(strSym))


name.x,name.y,strSym,user_code.x,user_code.y
<chr>,<chr>,<dbl>,<chr>,<chr>
ALBANA,Spessa di Cividale,0.4074074,06N601,105
ALBANA,Prepotto,0.0000000,06N601,217500
ALESSO,Alesso,1.0000000,06C551,401500
AMPEZZO,Enemonzo,0.5238095,06C120,ENE
ANDRAZZA,Forni di Sopra,0.4702381,06C006,231300
ANDRAZZA,Forni di Sopra,0.4702381,06C006,FSP
ANDREIS PIAZZOLA CAMPEGGIO,Andreis,0.7564103,06A464,103600
ANDREIS PIAZZOLA CAMPEGGIO,Barcis,0.6068376,06A464,103400
ANDREIS PIAZZOLA CAMPEGGIO,Pala Barzana,0.6059829,06A464,503500


In [ ]:
#  Non eseguire! Procedura per la riassegnazione dei metadati da OSMER
source("notebooks/ds_regionali/FVG/procedures/load_daily.R")
om <- query_checkpoint_meta("FVG", "merged", conns$data) |> collect()
nm <- load_osmer_meta()

assocs1 <- closest_within(om |> select(sensor_key, lon, lat), nm |> select(user_code, lon, lat), 170, conns$stations) |>
    group_by(sensor_key) |>
    slice_min(distance) |>
    arrange(desc(distance)) |>
    left_join(om |> select(sensor_key, name)) |>
    left_join(nm |> select(user_code, name), by = "user_code")

om1 <- om |> anti_join(assocs1, by = "sensor_key")
nm1 <- nm |> anti_join(assocs1, by = "user_code")

assocs1 |>
    group_by(sensor_key) |>
    filter(n() > 1L) |>
    nrow()
assocs1 |>
    group_by(user_code) |>
    filter(n() > 1L) |>
    nrow()

assocs2_ <- closest_within(om1 |> select(sensor_key, lon, lat), nm1 |> select(user_code, lon, lat), 800, conns$stations) |>
    group_by(sensor_key) |>
    slice_min(distance) |>
    arrange(desc(distance)) |>
    left_join(om1 |> select(sensor_key, name)) |>
    left_join(nm1 |> select(user_code, name), by = "user_code")

assocs2 <- bind_rows(assocs1, assocs2_)
assocs2 |>
    group_by(sensor_key) |>
    filter(n() > 1L) |>
    nrow()
assocs2 |>
    group_by(user_code) |>
    filter(n() > 1L) |>
    nrow()

om2 <- om1 |> anti_join(assocs2, by = "sensor_key")
nm2 <- nm1 |> anti_join(assocs2, by = "user_code")

remaining <- closest_within(om2 |> select(sensor_key, lon, lat), nm2 |> select(user_code, lon, lat), 10000, conns$stations) |>
    left_join(om2 |> select(sensor_key, name, elevation), by = "sensor_key") |>
    left_join(nm2 |> select(user_code, name, elevation), by = "user_code")
remaining |> write.xlsx(file.path("notebooks", "ds_regionali", "FVG", "remaining.xlsx"), asTable = TRUE)
assocs2 |> write.xlsx(file.path("notebooks", "ds_regionali", "FVG", "assocs2.xlsx"), asTable = TRUE)
# remaining and assocs2 are manually merged in the Excel file


In [ ]:
#  Lanciare in caso di regenerazione del dataset merged
# source("notebooks/ds_regionali/FVG/procedures/load_daily.R")
# old_meta <- query_checkpoint_meta(new_dataset, "merged", conns$data) |>
#     collect()
# new_meta <- load_osmer_meta() |> select(-series_first, -gestore)
# transition <- read.csv2(file.path("notebooks", "ds_regionali", "FVG", "pairing_codes.csv"), stringsAsFactors = FALSE)
# updated_metas <- old_meta |>
#     select(!all_of(colnames(new_meta))) |>
#     inner_join(transition, by = "sensor_key") |>
#     left_join(new_meta, by = "user_code")
# remaining_metas <- old_meta |> anti_join(updated_metas, by = "sensor_key")
# metas <- bind_rows(updated_metas, remaining_metas)
# write_parquet(metas, file.path("db", "metadata", "merged", "FVG", "part-0.parquet"))
